In [1]:
! pip install -U spacy -q

In [2]:
!python -m spacy info


============================== Info about spaCy ==============================

spaCy version    3.8.7                         
Location         /usr/local/lib/python3.12/dist-packages/spacy
Platform         Linux-6.6.105+-x86_64-with-glibc2.35
Python version   3.12.12                       
Pipelines        en_core_web_sm (3.8.0)        



In [3]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm

nlp = spacy.blank("en")
db = DocBin()

In [4]:
import json
f = open('/content/annotations.json')
TRAIN_DATA = json.load(f)

In [5]:
TRAIN_DATA

{'classes': ['ORG',
  'PEOPLE',
  'MONEY',
  'STOCK',
  'PERCENTAGE',
  'DATE',
  'COUNTRY',
  'FINANCIAL ENTITIES',
  'CONTINENT',
  'NATURAL NUMBERS',
  'EVENT',
  'PRODUCT',
  'TIME'],
 'annotations': [['Bitcoin’s price surged by 18% in the past week, breaking the $45,000 mark after news that Tesla Inc. (TSLA) has begun accepting cryptocurrency for vehicle purchases. The announcement sent shockwaves through the market, with other major companies like Square Inc. (SQ) and PayPal (PYPL) also signaling interest in integrating Bitcoin payments. Analysts predict that Bitcoin could rise another 10% by the end of Q4 2025 if institutional adoption continues to increase.\r',
   {'entities': [[0, 9, 'FINANCIAL ENTITIES'],
     [26, 29, 'PERCENTAGE'],
     [61, 68, 'MONEY'],
     [90, 107, 'ORG'],
     [128, 142, 'FINANCIAL ENTITIES'],
     [251, 267, 'ORG'],
     [272, 284, 'ORG'],
     [325, 332, 'FINANCIAL ENTITIES'],
     [392, 395, 'PERCENTAGE'],
     [410, 417, 'DATE']]}],
  ['\r', {'ent

In [10]:


for item in tqdm(TRAIN_DATA['annotations']):
   if item is None or len(item) != 2:
        continue
   text, annot = item
   doc = nlp.make_doc(text)
   ents = []
   for start, end, label in annot["entities"]:
       span = doc.char_span(start, end, label=label, alignment_mode="contract")
       if span is None:
           print("Skipping entity")
       else:
           ents.append(span)
   doc.ents = ents
   db.add(doc)
   db.to_disk("./training_data.spacy")


100%|██████████| 59/59 [00:00<00:00, 376.81it/s]


In [11]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency --force

⚠ To generate a more effective transformer-based config (GPU-only),
install the spacy-transformers package and re-run this command. The config
generated now does not use transformers.
ℹ Generated config template specific for your use case
- Language: en
- Pipeline: ner
- Optimize for: efficiency
- Hardware: CPU
- Transformer: None
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
!python -m spacy train config.cfg --output ./ --paths.train ./training_data.spacy --paths.dev ./training_data.spacy

ℹ Saving to output directory: .
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     48.24    0.00    0.00    0.00    0.00
  6     200       1419.42   3135.25   74.07   74.07   74.07    0.74
 12     400        489.31    758.33   94.80   95.15   94.44    0.95
 22     600        151.04    293.44   98.33   98.51   98.15    0.98
 34     800         72.39     98.52   99.45   99.26   99.63    0.99
 50    1000         87.58     82.24   99.63   99.63   99.63    1.00
 69    1200         67.94     56.58   99.63   99.63   99.63    1.00
 94    1400         61.83     63.11   99.63   99.63   99.63    1.00
124    1600        163.55     89.96 

In [13]:
nlp_ner = spacy.load("/content/model-best")

In [14]:
doc = nlp_ner("Market analysts at MorganEast Research, based in Singapore, forecast that the Indian digital lending sector will surpass ₹1.8 lakh crore in transaction volume by 2027, with Aurora capturing approximately 6.4% market share.")

In [15]:
spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter